In [1]:
from pathlib import Path
from gnpy.tools.json_io import load_equipment, load_network
from gnpy.core.utils import lin2db, automatic_nch
from gnpy.core.elements import Transceiver, Roadm, Fiber, Edfa
from gnpy.tools.worker_utils import designed_network, transmission_simulation
import networkx as nx
import sys
import traceback

In [ ]:
# === 1) PARÁMETROS DEL CANAL ÓPTICO (editables desde el código) ===
f_min = 191.3e12             # Hz
f_max = 195.1e12             # Hz
spacing = 50e9               # Hz
baud_rate = 32e9          # Hz
roll_off = 0.15              # sin unidad
tx_osnr = 35                # dB
sys_margins = 2              # dB
power_dbm = 2                # dBm
power_range_db = [0, 0, 1]   # [inicio, fin, paso] en dB

# === Archivos ===
EQPT = Path("eqpt_configv1.json")
TOPO = Path("Sweden_OpenROADMv4_example_network.json")

def tipo(n):
    if isinstance(n, Transceiver): return "TX/RX"
    if isinstance(n, Roadm): return "Roadm"
    if isinstance(n, Fiber): return "Fiber"
    if isinstance(n, Edfa): return "EDFA"
    return "Otro"

# === Entrada del usuario ===
try:
    num_rutas = int(input("Número de rutas a calcular: ").strip())
    if num_rutas < 1:
        raise ValueError
except ValueError:
    sys.exit(" Entradas inválidas. Debes ingresar un número válido.")

# === Cargar red y modificar parámetros del canal óptico ===
equipment = load_equipment(EQPT)
network = load_network(TOPO, equipment)

# Sobrescribir los parámetros del canal óptico
si = list(equipment["SI"].values())[0]
si.f_min = f_min
si.f_max = f_max
si.spacing = spacing
si.baud_rate = baud_rate
si.roll_off = roll_off
si.tx_osnr = tx_osnr
si.sys_margins = sys_margins
si.power_dbm = power_dbm
si.power_range_db = power_range_db

# Calcular número de canales
num_channels = automatic_nch(f_min, f_max, spacing)

# Mostrar parámetros en consola
print("\nPARÁMETROS DEL CANAL ÓPTICO")
print(f"  f_min:           {f_min/1e12:.2f} THz")
print(f"  f_max:           {f_max/1e12:.2f} THz")
print(f"  spacing:         {spacing/1e9:.2f} GHz")
print(f"  num_channels:    {num_channels}")
print(f"  baud_rate:       {baud_rate/1e9:.2f} Gbaud")
print(f"  roll_off:        {roll_off}")
print(f"  tx_osnr:         {tx_osnr} dB")
print(f"  sys_margins:     {sys_margins} dB")
print(f"  power_dbm:       {power_dbm} dBm")
print(f"  power_range_db:  {power_range_db}")

uid2node = {n.uid: n for n in network.nodes()}

# Transceivers disponibles
tx_nodes = [n for n in network.nodes() if isinstance(n, Transceiver)]
tx_uids = [n.uid for n in tx_nodes]

print("\n Transceivers disponibles:")
for i, uid in enumerate(tx_uids):
    print(f"  {i + 1}. {uid}")

src_uid = input("Ingrese el UID del transceiver origen (TX): ").strip()
if src_uid not in tx_uids:
    sys.exit(f" Transceiver '{src_uid}' no encontrado.")

dst_uids = [uid for uid in tx_uids if uid != src_uid]
print("\n Transceivers destino disponibles:")
for i, uid in enumerate(dst_uids):
    print(f"  {i + 1}. {uid}")
dst_uid = input("Ingrese el UID del transceiver destino (RX): ").strip()
if dst_uid not in dst_uids:
    sys.exit(f" Transceiver '{dst_uid}' no válido o igual al origen.")

src, dst = uid2node[src_uid], uid2node[dst_uid]

# Crear grafo para rutas
G = nx.DiGraph()
for u, v in network.edges():
    G.add_edge(u.uid, v.uid)

print(f"\n Buscando hasta {num_rutas} rutas entre {src.uid} → {dst.uid}...\n")

# Buscar rutas simples
paths_uid = list(nx.shortest_simple_paths(G, src_uid, dst_uid))[:num_rutas]

# Evaluar rutas
resultados = []
for i, uid_path in enumerate(paths_uid):
    try:
        path_nodes = [uid2node[uid] for uid in uid_path]

        net_designed, req, ref_req = designed_network(
            equipment, network,
            source=src.uid,
            destination=dst.uid,
            nodes_list=[n.uid for n in path_nodes],
            loose_list=['STRICT'],
            args_power=power_dbm
        )

        path, _, _, infos = transmission_simulation(equipment, net_designed, req, ref_req)
        receiver = path[-1]

        # Copiar métricas desde infos hacia el receptor (para compatibilidad)
        if hasattr(infos, "snr"):
            receiver.snr = infos.snr
        if hasattr(infos, "osnr_ase"):
            receiver.osnr_ase = infos.osnr_ase

        if hasattr(receiver, "snr_01nm") and hasattr(receiver, "osnr_ase_01nm"):
            dist_total = sum(getattr(n, 'length', getattr(getattr(n, 'params', None), 'length', 0))
                             for n in path if isinstance(n, Fiber))

            resultados.append({
                'ruta': path,
                'receiver': receiver,
                'snr_01nm': receiver.snr_01nm.mean(),
                'snr_bw': receiver.snr.mean() if hasattr(receiver, "snr") else None,
                'osnr_01nm': receiver.osnr_ase_01nm.mean(),
                'osnr_bw': receiver.osnr_ase.mean() if hasattr(receiver, "osnr_ase") else None,
                'dist': dist_total
            })
        else:
            print(f"⚠️  Ruta {i+1} falló: el receptor no tiene snr_01nm/osnr_ase_01nm.")
            continue

    except Exception as e:
        print(f" Ruta {i+1} falló: {e}")
        traceback.print_exc()

# Ordenar resultados: mejor SNR (0.1nm)
resultados.sort(key=lambda r: (-r['snr_01nm'], r['dist']))

# Mostrar resultados
for idx, res in enumerate(resultados):
    ruta = res['ruta']
    receiver = res['receiver']
    print(f"\n Ruta {idx + 1}")
    print("Tabla de nodos en la ruta:")
    print("Idx | {:>30} | {:<10}".format("UID", "Tipo"))
    print("-" * 50)
    for i, n in enumerate(ruta):
        print("{:3d} | {:>30} | {:<10}".format(i, n.uid, tipo(n)))

    print("\nDetalles de fibras:")
    for node in ruta:
        if isinstance(node, Fiber):
            length = getattr(node, 'length', getattr(node.params, 'length', 0))
            print(f"  {node.uid}: {length / 1000:.2f} km")

    print(f"\n Receptor final: {receiver.uid}")
    print(f"  SNR (0.1nm, dB):             {res['snr_01nm']:.2f}")
    print(f"  SNR (signal bw, dB):         {res['snr_bw']:.2f}"       if res['snr_bw'] else "  SNR (signal bw, dB):         N/A")
    print(f"  OSNR ASE (0.1nm, dB):        {res['osnr_01nm']:.2f}")
    print(f"  OSNR ASE (signal bw, dB):    {res['osnr_bw']:.2f}"     if res['osnr_bw'] else "  OSNR ASE (signal bw, dB):    N/A")
    print(f"\n Distancia total recorrida:   {res['dist'] / 1000:.2f} km")



	WARNING missing type_variety attribute in eqpt_config.json[Roadm]
	default value is type_variety = default




PARÁMETROS DEL CANAL ÓPTICO
  f_min:           191.30 THz
  f_max:           195.10 THz
  spacing:         50.00 GHz
  baud_rate:       32.00 Gbaud
  roll_off:        0.15
  tx_osnr:         35 dB
  sys_margins:     2 dB
  power_dbm:       2 dBm
  power_range_db:  [0, 0, 1]

 Transceivers disponibles:
  1. trx_Stockholm
  2. trx_Gothenburg
  3. trx_Malmö
  4. trx_Uppsala
  5. trx_Västerås
  6. trx_Örebro
  7. trx_Linköping
  8. trx_Helsingborg
  9. trx_Jönköping
  10. trx_Umeå
  11. trx_Norrköping
  12. trx_Karlstad
  13. trx_Sundsvall
  14. trx_Gävle
  15. trx_Borås

 Transceivers destino disponibles:
  1. trx_Gothenburg
  2. trx_Malmö
  3. trx_Uppsala
  4. trx_Västerås
  5. trx_Örebro
  6. trx_Linköping
  7. trx_Helsingborg
  8. trx_Jönköping
  9. trx_Umeå
  10. trx_Norrköping
  11. trx_Karlstad
  12. trx_Sundsvall
  13. trx_Gävle
  14. trx_Borås

 Buscando hasta 2 rutas entre trx_Stockholm → trx_Uppsala...


 Ruta 1
Tabla de nodos en la ruta:
Idx |                            UID | 